<h1 align='center'>BitCoin Data Transformation Internal</h1>

In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt

client = MongoClient("129.26.78.40:27017")
db = client['BitCoin']

In [3]:
db['blocks_2_aggregation'].find_one()

{u'_id': {u'reciever': u'112BDjNgWGwRyH1yr9W2jwQ96KQvZUPrQ2',
  u'sender': u'19Cu4qFyu5CE2hNRBpPrir6w5svYNA7tFZ',
  u'time': 1485103761,
  u'tx_hash': u'00000000000000000000cb4dad1c3b1076a9a71e5f21196e2908f4a50ac3ddad'},
 u'amount': 571538,
 u'tx_time': datetime.datetime(2017, 1, 22, 16, 49, 21)}

In [7]:
for a in db['blocks_2_aggregation'].find().sort([('tx_time',-1)]).limit(1):
    print a

{u'amount': 180000, u'_id': {u'tx_hash': u'000000000000000001417e1d867a8e5c51284c1896c2f3474a3df6062a13ba2f', u'time': 1485999334, u'sender': u'1JugUq597mSdE2a4nVNTVuhMwHYKjCCxpz', u'reciever': u'1H1HjD8qUbENzXcWF9SFut6g66pqYH4M9f'}, u'tx_time': datetime.datetime(2017, 2, 2, 1, 35, 34)}


In [8]:
for a in db['blocks_2_aggregation'].find().sort([('tx_time',1)]).limit(1):
    print a

{u'amount': 150000, u'_id': {u'tx_hash': u'00000000000000000341301ddd43d60a708cc920312942220e1b4f2b9a9131ed', u'time': 1484711444, u'sender': u'1KfKNpxtUc77FkFg5fCZPQSsXUdSdLv2np', u'reciever': u'1L39j9Eot75v6ngVTwDpztT64hSJy3V1W8'}, u'tx_time': datetime.datetime(2017, 1, 18, 3, 50, 44)}


## Aggregate the BitCoin amount per transaction per address

In [ ]:
for i in range(1, 17):
    db["blocks_{0}".format(i)].aggregate([{"$project":
                                      {"block_index":"$block_index",
                                       "_id":0,
                                       "height":"$block_height",
                                       "relayed_by":"$relayed_by",
                                       "hash": "$hash",
                                       "tx":"$tx"}},
                                 {"$unwind":"$tx"},
                                 {"$project":
                                         {"inputs":"$tx.inputs",
                                          "out":"$tx.out",
                                          "time":"$tx.time",
                                          "hash": "$hash",
                                          "_id":0
                                         }},
                                 {"$unwind":"$inputs"},
                                 {"$project":
                                            {"time":"$time",
                                             "hash": "$hash",
                                             "address_source":"$inputs.prev_out.addr",
                                             "amount":"$inputs.prev_out.value",
                                             "out":"$out"}},
                                 {"$unwind":"$out"},                             
                                 {"$group": 
                                          {
                                            "_id": {"tx_hash": "$hash", "sender":"$address_source", "time": "$time", "reciever":"$out.addr", "tag":"$out.addr_tag_link"},                                                                                     
                                                "amount":{ "$sum": "$out.value"}                                   
                                                
                                            }
                                 },
                                {"$match": { "amount": {"$gt": 0}}},                                
                                {"$out":"blocks_{0}_aggregation".format(i)}],allowDiskUse=True)

## Collect all aggrefation into one collection

In [ ]:
for i in range(1, 13):
    collection = "blocks_{0}_aggregation".format(i)
    data = db[collection].find({})
    db["blocks_aggregation"].insert_many(data)

## Group Data Hourly

In [90]:
for i in range(1, 13):
    db["blocks_{0}_aggregation".format(i)].aggregate([
                                 {"$group": {
                                     "_id": {"year": {"$year": "$tx_time" }, 
                                             "month": {"$month": "$tx_time"}, 
                                             "week": {"$week": "$tx_time"}, 
                                             "day": {"$dayOfMonth": "$tx_time"}, 
                                             "hour": {"$hour": "$tx_time"},
                                             "minutes": {"$minute":"$tx_time"},
                                             "second": {"$second":"$tx_time"}},
                                     "txs": {"$addToSet": {
                                            "h":"$_id.tx_hash",
                                            "s": "$_id.sender",
                                            "r": "$_id.reciever",
                                            "a": "$amount",                                          
                                            "t": "$tx_time"
                                     }}
                                 }},
                                
                                 {"$out":"blocks_{0}_aggregation_hourly".format(i)}],allowDiskUse=True)

OperationFailure: insert for $out failed: { connectionId: 183, err: "BSONObj size: 19116450 (0x123B1A2) is invalid. Size must be between 0 and 16793600(16MB) First element: _id: { week: 4, hour: 20, month: 1, second: 28...", code: 10334, n: 0, ok: 1.0 }

## Split the senders weekly

In [ ]:
db["blocks_aggregation"].aggregate([ {"$match": { "_id.time": {"$gt": mktime(datetime(2017, 1, 14).timetuple()), "$lte": mktime(datetime(2017, 1, 21).timetuple())}}},
                    {"$group": { 
                                 "_id": "$_id.sender", 
                                  "tx": {"$addToSet": {"hash":"$_id.tx_hash",
                                                       "receiver": "$_id.reciever",
                                                       "time": "$_id.time",
                                                       "amount": "$amount"}
                                        },
                                   "ntx": {"$sum": 1}
                                            }
                                },
                              {"$out":"senders_week_3"}
                             ],
                        allowDiskUse=True)

In [ ]:
print a